#### Starting with profiling

In [2]:
import pstats
p = pstats.Stats('profile.txt')
p.sort_stats('cumulative').print_stats(10)

Mon Jun  2 13:59:27 2025    profile.txt

         35967086 function calls (35833582 primitive calls) in 47.472 seconds

   Ordered by: cumulative time
   List reduced from 10429 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   4449/1    0.153    0.000   47.478   47.478 {built-in method builtins.exec}
        1    0.097    0.097   47.478   47.478 profiling.py:1(<module>)
     3006    0.017    0.000   22.665    0.008 /Users/krishuagarwal/Desktop/Programming/python/mlOps/mloops/lib/python3.10/site-packages/torch/utils/data/dataloader.py:728(__next__)
     3006    0.123    0.000   22.528    0.007 /Users/krishuagarwal/Desktop/Programming/python/mlOps/mloops/lib/python3.10/site-packages/torch/utils/data/dataloader.py:787(_next_data)
     3001    0.010    0.000   22.323    0.007 /Users/krishuagarwal/Desktop/Programming/python/mlOps/mloops/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py:47(fetch)
     3001    0.120    0.000

So basically : 
- `__next__` is called around `3006` times and per call takes around = 0.008 -> From the dataloader.
- `_next_data` is called `3006` times and per call takes around = 0.007 -> From the dataloader.

So the iter methods are being called a lot -> Interesting. 

`totime` : total time spent in functions excluding time spend in subfunctions

`cumtime` : total time including time spend in sub functions

#### Logging
`loguru` is being used.

In [6]:
from loguru import logger
logger.info("Profile stats printed successfully.")

# some examples of logging
logger.debug("Used for debugging your code.")
logger.info("Informative messages from your code.")
logger.warning("Everything works but there is something to be aware of.")
logger.error("There's been a mistake with the process.")
logger.critical("There is something terribly wrong and process may terminate.")

2025-06-02 14:31:01.425 | INFO     | __main__:<module>:2 - Profile stats printed successfully.
2025-06-02 14:31:01.426 | DEBUG    | __main__:<module>:5 - Used for debugging your code.
2025-06-02 14:31:01.427 | INFO     | __main__:<module>:6 - Informative messages from your code.
2025-06-02 14:31:01.428 | WARNING  | __main__:<module>:7 - Everything works but there is something to be aware of.
2025-06-02 14:31:01.429 | ERROR    | __main__:<module>:8 - There's been a mistake with the process.
2025-06-02 14:31:01.429 | CRITICAL | __main__:<module>:9 - There is something terribly wrong and process may terminate.


In [ ]:
import sys
from loguru import logger
logger.remove()  # Remove the default logger
logger.add(sys.stdout, level="INFO")  # Add a new logger with WARNING level

logger.add("my_log.log", level="INFO", rotation="100MB") # It is very easy to log to a file now, rotates it when it reaches 100MB
logger.debug("Used for debugging your code.")
logger.info("Informative messages from your code.")
logger.warning("Everything works but there is something to be aware of.")
logger.error("There's been a mistake with the process.")
logger.critical("There is something terribly wrong and process may terminate.")

2025-06-02 14:32:44.603 | INFO     | __main__:<module>:8 - Informative messages from your code.
2025-06-02 14:32:44.608 | WARNING  | __main__:<module>:9 - Everything works but there is something to be aware of.


2025-06-02 14:32:44.612 | ERROR    | __main__:<module>:10 - There's been a mistake with the process.
2025-06-02 14:32:44.653 | CRITICAL | __main__:<module>:11 - There is something terribly wrong and process may terminate.


- `logger.catch()` : for catching and sending errors -> can work with try and catch block
- `logger.format()` : for formatting the logging files.



#### Experimental Logging

In [18]:
import random

import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="nothing",
    # Set the wandb project where this run will be logged.
    project="learning",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

CommError: failed to upsert bucket: returned error 403: {"data":{"upsertBucket":null},"errors":[{"message":"permission denied","path":["upsertBucket"],"extensions":{"code":"PERMISSION_ERROR"}}]}